In [3]:
#Import biopython module to open FASTA files
from Bio import SeqIO

In [2]:
H20_MASS = 18.010565


In [5]:
# Monoisotopic mass
monoAminoMass =  {
                        'A': 71.03711,
                        'R': 156.10111,
                        'N': 114.0493,
                        'D': 115.02694,
                        'C': 103.00919,
                        'E': 129.04259,
                        'Q': 128.05858,
                        'G': 57.02146,
                        'H': 137.05891,
                        'I': 113.08406,
                        'L': 113.08406,
                        'K': 128.09496,
                        'M': 131.04049,
                        'F': 147.06841,
                        'P': 97.05276,
                        'S': 87.03203,
                        'T': 101.04768,
                        'W': 186.07931,
                        'Y': 163.06333,
                        'V': 99.06841,
                        
                        
                    }

In [6]:
"""Inputs: peptide string, max length of split peptide. 
   Outputs: all possible splits that could be formed that are smaller in length than the maxed input """
def splitDictPeptide(peptide, maxed):
    length = len(peptide)
    # splits will hold all possible splits that can occur
    splits = []
    # splitRef will hold a direct reference to the characters making up each split string: for starting peptide ABC,
    # the split AC = [0,2] 
    splitRef = []
    
    # imbedded for loops build all possible splits
    for i in range(0, length):
        character = peptide[i]
        toAdd=""
        # add and append first character and add and append reference number which indexes this character
        toAdd+=character
        splits.append(toAdd)
        ref = []
        ref.append(i)
        temp = list(ref)  # use list because otherwise shared memory overwrites
        splitRef.append(temp)
        
        # iterates through every character after current and adds it to the most recent string if max size
        # requirement is satisfied
        for j in range(i+1, length):
            toAdd+=peptide[j]
            if(maxSize(toAdd, maxed)):
                
                splits.append(toAdd)
                ref.append(j)
                temp = list(ref)
                
                splitRef.append(temp)
                temp=[]
      
      
        ref = []
        
        
    return splits, splitRef

In [7]:
"""Input: splits: list of splits, splitRef: list of the character indexes for splits, mined/maxed: min and max
   size requirements, overlapFlag: boolean value true if overlapping combinations are undesired.
   Output: all combinations of possible splits which meets criteria"""
def combineOverlapPeptide(splits, splitRef, mined, maxed, overlapFlag):
    
    # initialise combinations array to hold the possible combinations from the input splits
    combine = []
   
    # iterate through all of the splits and build up combinations which meet min/max/overlap criteria
    for i in range(0, len(splits)):
        if(minSize(splits[i], mined)):
            #Add linear peptide, can include option if necessary
            combine.append(splits[i])
        
        # toAdd holds temporary for insertion in final matrix if it meets criteria
        toAdd=""
        
        for j in range(i+1, len(splits)): 
            # create forward combinaiton of i and j
            toAdd+=splits[i]
            toAdd+=splits[j]
    
        # look to combine all checks together in a future for clarity
            if(maxSize(toAdd, maxed) and minSize(toAdd, mined)):
            
                if(overlapFlag==True):
                    if(overlapComp(splitRef[i],splitRef[j])):
                        combine.append(toAdd)
                else:
                    combine.append(toAdd)
               
            #create backwards combination of i and j
            toAdd = ""
            toAdd+=splits[j]
            toAdd+=splits[i]
            
            if(maxSize(toAdd, maxed) and minSize(toAdd, mined)):
                if(overlapFlag==True):
                    if(overlapComp(splitRef[i],splitRef[j])):
                        combine.append(toAdd)
                else:
                    combine.append(toAdd)
               
                
                
        
            toAdd=""
    return combine

In [8]:
# ensures length of split is smaller than or equal to max
def maxSize(split, maxed):
    if(len(split)>=maxed):
        return False
    return True

In [9]:
# ensures length of split is greater than min
def minSize(split, mined):
    if(len(split)<mined):
        return False
    return True

In [10]:
# checks if there is an intersection between two strings. Likely input it the splitRef data. 
# Outputs True if no intersection
def overlapComp(ref1,ref2):
    S1 = set(ref1)
    S2 = set(ref2)
    if(len(S1.intersection(S2))==0):
        return True
    return False

In [11]:
# opens FASTA file
def addSequenceList(input_file):
    
    fasta_sequences = SeqIO.parse(open(input_file),'fasta')
    sequenceDictionary = {}
    for fasta in fasta_sequences:
        name, sequence = fasta.id, fasta.seq.tostring()
        sequenceDictionary[name] = sequence
    return sequenceDictionary

In [12]:
# Adapted from https://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-whilst-preserving-order
# removes duplicates given a list
def removeDups(seq):
    seen = set()
    seen_add = seen.add
    initial = [x for x in seq if not (x in seen or seen_add(x))]
    temp = []
    # Remove permutations
    for i in range(0, len(initial)):
        for j in range(i+1, len(initial)):
            if(combRemove(initial[i], initial[j])):
                temp.append(initial[j])
        
    final = [x for x in initial if x not in temp]
    return final


In [13]:
"""
Returns true if two strings are a permutation of each other -> Called in removeDups
"""
def combRemove(ref1, ref2):
    return (len(ref1) == len(ref2) and sorted(ref1) == sorted(ref2))

In [14]:
# combines an array of strings into one string. Used for ultimately segments from multiple peptides
def combinePeptides(peptideList):
    finalPeptide = ''.join(peptideList)
    return finalPeptide


In [15]:
# generates most of the permutations possible when switching from A to a in all strings originally containing an A
# input is a list of all combinations
def modTest(combine):
    # A, B, C  convert to a, b, c
    modComb = []
    for string in combine:
        if 'A' in string:
            
            numOccur = string.count('A')
            
            for i in range(0, numOccur):
                temp = string
                temp = temp.replace("A","a", i+1)
               
                modComb.append(temp)
    return modComb

In [16]:
def combMass(combine):
    massDict = {}
    for combination in combine:
        totalMass = 0
        for character in combination:
            totalMass += monoAminoMass[character]
        totalMass+=H20_MASS
        massDict[combination] = totalMass   
    return massDict

In [17]:
maxed = 12
mined= 0
overlap = True
splits, splitRef = splitDictPeptide("ARDCE", maxed)

splits = removeDups(splits)





In [19]:
combine1 = ['AR']
print(combMass(combine1))

{'AR': 245.14878499999998}


In [252]:
print(splits)
print(len(splits))

['A', 'AR', 'ARD', 'ARDC', 'ARDCE', 'R', 'RD', 'RDC', 'RDCE', 'D', 'DC', 'DCE', 'C', 'CE', 'E']
15


In [253]:
combine = combineOverlapPeptide(splits, splitRef, mined, maxed, overlap)
combine = removeDups(combine)

print(combine)
print(len(combine))

massDict = combMass(combine)
print(massDict)

['A', 'AR', 'ARD', 'ARDC', 'ARDCE', 'AD', 'ADC', 'ADCE', 'AC', 'ACE', 'AE', 'ARC', 'ARCE', 'ARE', 'ARDE', 'R', 'RD', 'RDC', 'RDCE', 'RC', 'RCE', 'RE', 'RDE', 'D', 'DC', 'DCE', 'DE', 'C', 'CE', 'E']
30
{'A': 71.03711, 'AR': 227.13822, 'ARD': 342.16516, 'ARDC': 445.17435, 'ARDCE': 574.21694, 'AD': 186.06405, 'ADC': 289.07324, 'ADCE': 418.11582999999996, 'AC': 174.0463, 'ACE': 303.08889, 'AE': 200.0797, 'ARC': 330.14741, 'ARCE': 459.18999999999994, 'ARE': 356.18080999999995, 'ARDE': 471.20775000000003, 'R': 156.10111, 'RD': 271.12805000000003, 'RDC': 374.13724, 'RDCE': 503.17983000000004, 'RC': 259.1103, 'RCE': 388.15288999999996, 'RE': 285.14369999999997, 'RDE': 400.17064000000005, 'D': 115.02694, 'DC': 218.03613, 'DCE': 347.07872, 'DE': 244.06953, 'C': 103.00919, 'CE': 232.05178, 'E': 129.04259}


In [215]:
modcomb = modTest(combine)

print("Combine original")
print(combine)

#print("Modcomb")
#print(modcomb)



Combine original
['A', 'AB', 'BA', 'ABA', 'BAA', 'ABAA', 'BAAA', 'ABAAA', 'BAAAA', 'AAA', 'AAAA', 'AAAB', 'AAAAB', 'B', 'AAB', 'AA']
Modcomb
['a', 'aB', 'Ba', 'aBA', 'aBa', 'BaA', 'Baa', 'aBAA', 'aBaA', 'aBaa', 'BaAA', 'BaaA', 'Baaa', 'aBAAA', 'aBaAA', 'aBaaA', 'aBaaa', 'BaAAA', 'BaaAA', 'BaaaA', 'Baaaa', 'aAA', 'aaA', 'aaa', 'aAAA', 'aaAA', 'aaaA', 'aaaa', 'aAAB', 'aaAB', 'aaaB', 'aAAAB', 'aaAAB', 'aaaAB', 'aaaaB', 'aAB', 'aaB', 'aA', 'aa']


In [1]:
# taking FASTA dictionary and passing through our splits and combine functions
sequenceDictionary = addSequenceList("Example.fasta")
for key, value in sequenceDictionary.items():
    
    splits, splitRef = splitDictPeptide(value, maxed)
    
    splits = removeDups(splits)
    combine = combineOverlapPeptide(splits, splitRef, mined, maxed,  overlap)
    combine = removeDups(combine)
    
    print(len(splits))
    print(len(combine))
    
    break;

NameError: name 'addSequenceList' is not defined

Analysis of removing duplicates at split level and combined level:
    - Split level: 3.1 mil to 1.8 mil
    - Combined level: 1.8 mil to 1.6 mil
    